In [9]:
!pip install flask_ngrok
!pip install flask
!pip install openai
!pip install python-dotenv
!pip install line-bot-sdk
# 爬蟲要用的套件
!pip install beautifulsoup4
!pip install requests
!pip install phonenumbers

In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [58]:
query = "0917209216"
ans = phoneSearch(query)
ans

'小心!此電話可能為貸款推銷'

In [57]:
import requests
import phonenumbers
from bs4 import BeautifulSoup
def phoneSearch(query:str) -> str:
    # 電話型態     
    phoneType = {"無聲電話","廣告簡訊","想一聲就掛","詐騙簡訊","一接就掛","詐騙電話",
                 "貸款推銷","銀行推銷","保險推銷","投資推銷","騷擾電話"}
    pNum = '+886'+query
    try:
        pNum = phonenumbers.parse(pNum)
        set1 = set()
        if phonenumbers.is_valid_number(pNum):
            url = "https://whocall.cc/search/"+query
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
            response = requests.get(url, headers=headers)
            response.encoding = 'utf8'
            soup = BeautifulSoup(response.text, 'html.parser')
            tags = soup.find_all("p", {"class": "card-text"})
            for tag in tags:
                if tag.text in phoneType:
                    return "小心!此電話可能為"+tag.text
        else:
            return "非臺灣地區電話格式"
    except phonenumbers.NumberParseException:
        return "電話數字格式錯誤"
    except Exception as e:
        return str(e)
    return "查無此號或一般未知來電"

In [ ]:
import openai
from flask import Flask, request

# 載入 LINE Message API 相關函式庫
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage   # 載入 TextSendMessage 模組
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    print(json_data)
    try:
#         取得環境變數
        gpt_key=os.environ.get('CHATGPT')
        channel_token=os.environ.get('CHANNEL_TOKEN')
        channel_secret=os.environ.get('CHANNEL_SECRET')
        line_bot_api=LineBotApi(channel_token)
        handler=WebhookHandler(channel_secret)
        signature=request.headers['X-Line-Signature']
        handler.handle(body, signature)
        tk=json_data['events'][0]['replyToken']
        msg=json_data['events'][0]['message']['text']
        # 取出文字的前五個字元，轉換成小寫
        ai_msg = msg[:5].lower()
        reply_msg = ''
        # 取出文字的前五個字元是 chat:
        if ai_msg == 'chat ':
            openai.api_key = gpt_key
            # 將第五個字元之後的訊息發送給 OpenAI
            response = openai.Completion.create(
                model='text-davinci-003',
                prompt=msg[5:],
                max_tokens=256,
                temperature=0.5,
                )
            # 接收到回覆訊息後，移除換行符號
            reply_msg = response["choices"][0]["text"].replace('\n','')
        elif ai_msg == 'call ':
            query = msg[5:]
            reply_msg = phoneSearch(query)
        else:
            reply_msg = msg
        text_message = TextSendMessage(text=reply_msg)
        line_bot_api.reply_message(tk,text_message)
    except:
        print('error')
    return 'OK'

if __name__ == "__main__":
    app.run()  #最後生成網址，貼到line: messaging API-->webhook url

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'destination': 'Ue73009b34b87bd9fc36ee196aa196265', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '457262810754711858', 'text': 'call 0917209216'}, 'webhookEventId': '01H1M8QS431Y4G5RNB7Z6A320P', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1685381178065, 'source': {'type': 'user', 'userId': 'U03bbae7973739f2fe42eed377fd32992'}, 'replyToken': '66d72fbfb3dc4e9cbc7095dcbe6651f0', 'mode': 'active'}]}


127.0.0.1 - - [30/May/2023 01:28:57] "POST / HTTP/1.1" 200 -
